In [ ]:
# Imports necessary Packages
import numpy as np
import scipy.stats as sps
import pandas as pd

# Specific Plotting Packages
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import patches
from mpl_toolkits.mplot3d import Axes3D

# sklearn
from sklearn.preprocessing import normalize

# Import Custom Packages
import supplemental_funcs as sf

# to show in notebook
%matplotlib inline

In [ ]:
# set a random seed
np.random.seed(24513)

In [ ]:
plt.rcParams.update({'font.size': 14})

In [ ]:
# defines a general color pallette
col_highlight = 'xkcd:yellow'
col_blue = 'xkcd:sky'
col_red = 'xkcd:red orange'

In [ ]:
# save all figures dictionary
# fig_save_dictionary { 'filename' : figname }
fig_all_master = {}

# Stationary or Wobbly Plate?

Defines model map $Q:\Lambda\rightarrow\mathcal{D}$ as:

\begin{align}
Q(\lambda)=\begin{pmatrix}0.6 & 0.7 \\
0.8 & 0.6
\end{pmatrix}\cdot \begin{pmatrix} \lambda_1 \\ \lambda_2 \end{pmatrix} +3
\end{align}

where $\vec{x}_A=(0.6,0.7)$ and $\vec{x}_B=(0.8,0.6)$ are the two locations and $y_0=3$ is the height of the plate above the origin.

In [ ]:
n_obs = 500 # number of observations
locations = np.array([[0.6,0.7],[0.8,0.6]]) # location of observations
y0 = 3

In [ ]:
# mean of parameter slope or true fixed value for the slope
lam_mean = np.array([0.8,1.4])

# define the observed covariance
cov_rot = np.pi/6 # rotation in radians
cov_dir = np.array([[np.cos(cov_rot),-np.sin(cov_rot)],
                    [np.sin(cov_rot),np.cos(cov_rot)]])
cov_eig = 0.25*np.array([3,1])*np.eye(2)
obs_cov = np.linalg.multi_dot([cov_dir,cov_eig,cov_dir.T])

# define the observed distribution
obs_mean = np.dot(locations,lam_mean)+y0
obs_dist = sps.multivariate_normal(obs_mean,obs_cov)

# obtain inverse lambda distribution
loc_inverse = np.linalg.inv(locations)
lam_cov = np.linalg.multi_dot([loc_inverse,obs_cov,loc_inverse.T])
lam_target_dist = sps.multivariate_normal(lam_mean,lam_cov)

In [ ]:
obs_cov

### Plot Different Scenario Solutions

Here I plot the data space and the corresponding inverse solutions in the parameter space for the two different kinds of problems:

1. Parameter **Estimation** Problem
2. Parameter **Distribution** Problem

These correspond to a situation where the plate has a fixed slope (stationary plate) or has a random slope (wobbly plate), respectively.

In [ ]:
# get eigenvalues and eigenvectors of matrix
eig_val, eig_vec = np.linalg.eig(obs_cov)

In [ ]:
# make lam-sapce grid
lamx = np.linspace(-8, 8, 100)
lamy = np.linspace(-8, 8, 100)
lamX, lamY = np.meshgrid(lamx, lamy)
eval_lamXY = np.vstack([lamX.ravel(),lamY.ravel()])


# make data-space grid
qx = np.linspace(0, 6, 100)
qy = np.linspace(0, 6, 100)
qX, qY = np.meshgrid(qx, qy)
eval_qXY = np.vstack([qX.ravel(),qY.ravel()])

# eval points and reshape
Z1 = lam_target_dist.pdf(eval_lamXY.T).reshape(lamX.shape)
Z2 = obs_dist.pdf(eval_qXY.T).reshape(qX.shape)

In [ ]:
fig, (ax_lam,ax_dat) = plt.subplots(1,2)
fig.set_figwidth(8)
contour = ax_lam.contour(lamX,lamY,Z1)
ax_lam.set_title('Inverse Solution')
ax_lam.set_xlabel('$\lambda_1$')
ax_lam.set_ylabel('$\lambda_2$')
ax_lam.plot(lam_mean[0],lam_mean[1],'or',label='Fixed Slope')
contour.collections[-4].set_label('Random Slope')
ax_lam.legend()

ax_dat.contour(qX,qY,Z2)
ax_dat.annotate('',xy=obs_mean-2.5*eig_val[1]*eig_vec[:,1],xytext=obs_mean,
                arrowprops=dict(arrowstyle='->'))

ax_dat.annotate('',xy=obs_mean-2.5*eig_val[0]*eig_vec[:,0],xytext=obs_mean,
                arrowprops=dict(arrowstyle='->'))
ax_dat.set_title('Observed Height Distribution')
ax_dat.set_xlabel('$q_A$')
ax_dat.set_ylabel('$q_B$')


### Exact Update Function

In [ ]:
def check_nd_shape(lam,dim=1):
    if len(lam.shape)==0:
        vals = lam.reshape(1,1)
    elif len(lam.shape)==1:
        if dim==1:
            vals = lam.reshape(lam.shape[0],1)
        else:
            vals = lam.reshape(1,lam.shape[0])
    else:
        vals = lam
    return vals

def Qmap_linear(lam,A=np.eye(2),b=np.zeros(2),input_dim=2):
    vals = check_nd_shape(lam,dim=input_dim)
    output = np.dot(A,vals.T).T+b
    output = np.squeeze(output) # removes singleton dimensions
    return output

In [ ]:
class dci_update:
    def __init__(self,init,pred,obs,Qmap):
        self.init_dist = init
        self.pred_dist = pred
        self.obs_dist = obs
        self.Q = Qmap
        
    def pdf(self,lam):
        # specify vals to evaluate and get Q(lam)
        lam_vals = check_nd_shape(lam)
        lam_update = check_nd_shape(np.zeros(lam_vals.shape[0]))
        q = self.Q(lam_vals)
        
        # compute pdfs
        init_vals = check_nd_shape(self.init_dist.pdf(lam_vals))
        pred_vals = check_nd_shape(self.pred_dist.pdf(q))
        obs_vals = check_nd_shape(self.obs_dist.pdf(q))
        
        # predictability assumption
        nonzeros = init_vals != 0
        nonzero_denom = pred_vals > 1e-6*obs_vals 
        up_ind = np.logical_and(nonzeros,nonzero_denom)
        
        # return updated values
        lam_update[up_ind] = init_vals[up_ind]*obs_vals[up_ind]/pred_vals[up_ind]
        lam_update = np.squeeze(lam_update)
        
        return lam_update
        
    

# Update 2 to 2:

By 2-to-2, I mean that we are using the map as defined earlier:

$Q:\Lambda\rightarrow\mathcal{D}$ as:

\begin{align}
Q(\lambda)=\begin{pmatrix}0.6 & 0.7 \\
0.8 & 0.6
\end{pmatrix}\cdot \begin{pmatrix} \lambda_1 \\ \lambda_2 \end{pmatrix} +3
\end{align}

where $\vec{x}_A=(0.6,0.7)$ and $\vec{x}_B=(0.8,0.6)$ are the two locations and $y_0=3$ is the height of the plate above the origin.

In [ ]:
# define map
Q2to2 = lambda lam: Qmap_linear(lam, A=locations,b=y0)

# propose initial
initial_mean = np.array([-3,1]) # centered around the plate's slope being flat
initial_cov = 16*np.eye(2)  # value chosen to be very broad over the space
initial_prec = np.linalg.inv(initial_cov)
initial_dist = sps.multivariate_normal(initial_mean,initial_cov)

# construct predicted
predict_mean = np.dot(locations,initial_mean)+y0
predict_cov = np.linalg.multi_dot([locations,initial_cov,locations.T])
predict_prec = np.linalg.inv(predict_cov)
predict_dist = sps.multivariate_normal(predict_mean,predict_cov)

# remind myself of observed from previous
print(type(obs_dist))
print('Mean: ', obs_dist.mean)
print('Cov: ', obs_dist.cov)

# define update distribution class
update_dist_2D = dci_update(initial_dist,predict_dist,obs_dist,Q2to2)

In [ ]:
# compute elements of the picture
# in parameter space
Z_init = initial_dist.pdf(eval_lamXY.T).reshape(lamX.shape)
Z_up = update_dist_2D.pdf(eval_lamXY.T).reshape(lamX.shape)

# in data space
Z_pred = predict_dist.pdf(eval_qXY.T).reshape(qX.shape)
Z_obs = obs_dist.pdf(eval_qXY.T).reshape(qX.shape)

In [ ]:
# create basic DCI 4-plot
fig_dci_basic, axes = plt.subplots(2,2,subplot_kw={'aspect':'equal'})
fig_dci_basic.tight_layout(h_pad=1,w_pad=-1)
fig_dci_basic.set_figwidth(8)
fig_dci_basic.set_figheight(8)
Z_list = [[Z_init,Z_pred],[Z_up,Z_obs]]
title_list = [['Initial','Predicted'],['Update','Observed']]
for i,row_ax in enumerate(axes):
    for j,ax in enumerate(row_ax):
        if j==0:
            ax.contour(lamX,lamY,Z_list[i][j])
            ax.set_xlabel('$\lambda_1$')
            ax.set_ylabel('$\lambda_2$')
        else:
            ax.contour(qX,qY,Z_list[i][j])
            ax.set_xlabel('$y_A$')
            ax.set_ylabel('$y_B$')
        ax.set_title(title_list[i][j])

# figure name will be 
this_fig_name = 'fig_wobbly_dci_basic.png'
fig_all_master[this_fig_name] = fig_dci_basic

# # save just this fig
# fig_dci_basic.savefig('../'+this_fig_name)


# QoI 2 to 1:

Here we use only one of the sensors at location A. Thus the map is:

$Q:\Lambda\rightarrow\mathcal{D}$:

\begin{align}
Q(\lambda)=\begin{pmatrix}0.6 & 0.7
\end{pmatrix}\cdot \begin{pmatrix} \lambda_1 \\ \lambda_2 \end{pmatrix} +3
\end{align}

where $\vec{x}_A=(0.6,0.7)$ is the location of one of the sensors and $y_0=3$ is the height of the plate above the origin.

In [ ]:
# define the map
pointA = 0 # first sensor only.
Q2to1 = lambda lam: Qmap_linear(lam, A=locations[pointA],b=y0)

# use the same initial
print(type(initial_dist))
print('Mean: ', initial_dist.mean)
print('Cov: ', initial_dist.cov)

# define alternative initial dist
# initial term test
initial_mean_shift = initial_mean+np.array([0,-4])
initial_rotate = np.dot(1/np.sqrt(53)*np.diag([7,2]),initial_cov)
initial_cov_ROT = np.dot(initial_cov,1/np.sqrt(53)*np.diag([7,2]))
initial_dist_covbias = sps.multivariate_normal(initial_mean,initial_cov_ROT)

# define the new predicted
predict_mean_1D = Q2to1(initial_mean)
predict_cov_1D = np.linalg.multi_dot([locations[pointA],initial_cov,locations[pointA].T])
predict_prec_1D = 1/predict_cov_1D # now a scalar not a matrix
predict_dist_1D = sps.multivariate_normal(predict_mean_1D,predict_cov_1D)

# define the new observed distribution
obs_cov_1D = obs_cov[pointA,pointA] # just take the first sensor's variance
obs_dist_1D = sps.norm(obs_mean[pointA],np.sqrt(obs_cov_1D))
update_dist_1D = dci_update(initial_dist,predict_dist_1D,obs_dist_1D,Q2to1)


# define rotated 1D update
predict_cov_1D_ROT = np.linalg.multi_dot([locations[pointA],initial_cov_ROT,locations[pointA].T])
predict_dist_1D_ROT = sps.multivariate_normal(predict_mean_1D,predict_cov_1D_ROT)
update_dist_1D_ROT = dci_update(initial_dist_covbias,predict_dist_1D_ROT,obs_dist_1D,Q2to1)

In [ ]:
# compute elements of the picture
# in parameter space
print('Initial Dist Shape: ', Z_init.shape) # use same initial
Z_up_2to1 = update_dist_1D.pdf(eval_lamXY.T).reshape(lamX.shape)

# in data space
Z_pred_1D = predict_dist_1D.pdf(qx)
Z_obs_1D = obs_dist_1D.pdf(qx)

# rotated values
Z_init_ROT = initial_dist_covbias.pdf(eval_lamXY.T).reshape(lamX.shape)
Z_pred_1D_ROT = predict_dist_1D_ROT.pdf(qx)
Z_up_2to1_ROT = update_dist_1D_ROT.pdf(eval_lamXY.T).reshape(lamX.shape)

In [ ]:
# create basic DCI 4-plot
fig_dci_1D, axes = plt.subplots(2,2)
fig_dci_1D.tight_layout(h_pad=1,w_pad=1)
fig_dci_1D.set_figwidth(8)
fig_dci_1D.set_figheight(8)
Z_list = [[Z_init,Z_pred_1D],[Z_up_2to1,Z_obs_1D]]
title_list = [['Initial','Predicted'],['Update','Observed']]
for i,row_ax in enumerate(axes):
    for j,ax in enumerate(row_ax):
        if j==0:
            ax.contour(lamX,lamY,Z_list[i][j])
            ax.set_xlabel('$\lambda_1$')
            ax.set_ylabel('$\lambda_2$')
            
            # this is the exact solution to the invertible case
            if i==1:
                ax.plot(lam_mean[0],lam_mean[1],'ro')
                contour = ax.contour(lamX,lamY,Z_up)
                for k,collection in enumerate(contour.collections):
                    if k==2:
                        collection.set_visible(True)
                        collection.set_color('r')
                        collection.set_zorder(-1)
                    else:
                        collection.set_visible(False)

                
        else:
            ax.plot(qx,Z_list[i][j])
            ax.set_xlabel('$q$')
        ax.set_title(title_list[i][j])

# figure name will be 
this_fig_name = 'fig_wobbly_dci_2to1.png'
fig_all_master[this_fig_name] = fig_dci_1D

# save just this fig
# fig_dci_1D.savefig('../'+this_fig_name)


In [ ]:
# create basic DCI 4-plot
fig_dci_1D_covbias, axes = plt.subplots(2,2,subplot_kw={'aspect':'equal'})
fig_dci_1D_covbias.tight_layout(h_pad=1,w_pad=-1)
fig_dci_1D_covbias.set_figwidth(8)
fig_dci_1D_covbias.set_figheight(8)
Z_list = [[Z_init_ROT,Z_up_2to1_ROT],[Z_init,Z_up_2to1]]
title_list = [['Initial: Bias Cov','Update'],['Initial: Spherical Cov','Update']]
for i,row_ax in enumerate(axes):
    for j,ax in enumerate(row_ax):
        ax.contour(lamX,lamY,Z_list[i][j])

        # this is the exact solution to the invertible case
        if j==1:
            ax.plot(lam_mean[0],lam_mean[1],'ro')
            contour = ax.contour(lamX,lamY,Z_up)
            for k,collection in enumerate(contour.collections):
                if k==2:
                    collection.set_visible(True)
                    collection.set_color('r')
                    collection.set_zorder(-1)
                else:
                    collection.set_visible(False)
        ax.set_xlabel('$\lambda_1$')
        if j==0:
            ax.set_ylabel('$\lambda_2$')
        ax.set_title(title_list[i][j])

# # figure name will be 
this_fig_name = 'fig_wobbly_dci_bias_cov.png'
fig_all_master[this_fig_name] = fig_dci_1D_covbias

# # save just this figure
# fig_dci_1D_covbias.savefig('../'+this_fig_name)


### Eigenvector Analysis for 2-to-1 Linear Maps

In [ ]:
# general formulas for 2-to-1
XTX = np.outer(locations[pointA].T,locations[pointA])

q_cov_diff = 1/predict_cov_1D*(1-obs_cov_1D/predict_cov_1D)
update_cov = initial_cov-q_cov_diff*np.linalg.multi_dot([initial_cov,XTX,initial_cov])

q_mean_diff = obs_mean[pointA]-predict_mean_1D 
update_mean = initial_mean+np.dot(initial_cov,locations[pointA].T)*1/predict_cov_1D*q_mean_diff

# confirm that specific formulas for spherical cov are correct
# i.e., did I do the algebra right?

# check the exact eigendecomposition 
XXT = np.dot(locations[pointA],locations[pointA].T)
DxtxA, Uxtx = np.linalg.eig(XTX) # eigendecomposition of XTX using numpy
Dxtx = np.array([0,XXT]) # exact eigenvalues by hand
print('Check Eigenvalues: ',DxtxA,Dxtx, '\n') 
# these agree with each other with some roundoff error

Dup =  initial_cov[0,0]*(np.eye(2)-np.diag(Dxtx)/XXT)+obs_cov_1D/XXT*np.diag(Dxtx)/XXT
# print(Dup)
update_cov_SP = np.linalg.multi_dot([Uxtx,Dup,Uxtx.T])
print('Check Eigendecomposition: ')
print(update_cov)
print(update_cov_SP) #
print()

debias = np.dot(np.linalg.multi_dot([Uxtx,np.diag([0,1]),Uxtx.T]),initial_mean)
data_shift = 1/XXT*locations[pointA].T*(obs_mean[pointA]-y0)
update_mean_SP = initial_mean-debias+data_shift
print('Check Update Means: ')
print(update_mean,update_mean_SP)


[Normal Error Ellipses Tutorial](https://www.visiondummy.com/2014/04/draw-error-ellipse-representing-covariance-matrix/)


In [ ]:
fig_dci_decomp, (ax1,ax2) = plt.subplots(1,2,subplot_kw={'aspect':'equal'})
fig_dci_decomp.set_figwidth(8)
ax1.set_xlim(lamx[0],lamx[-1])
ax1.set_ylim(lamy[0],lamy[-1])

# initial ellipse
area90 = 4.605 # sets an area for the circles that includes 90% of data
init_center = initial_mean
init_width = 2*np.sqrt(area90/2*initial_cov[0,0])
init_ellipse = patches.Ellipse(xy=init_center,
                               width=init_width,height=init_width,
                               edgecolor='b',fc="None")

# update ellipse (using same mean)
up_width = 2*np.sqrt(area90/2*Dup[0,0])
up_height = 2*np.sqrt(area90/2*Dup[1,1])
rot_angle = np.arctan(Uxtx[1,0]/Uxtx[0,0])*180/np.pi

up_ellipse = patches.Ellipse(xy=init_center,angle=rot_angle,
                               width=up_width,height=up_height,
                               edgecolor='b',fc="None")

ax1.add_artist(init_ellipse)
ax1.add_artist(up_ellipse)

eig1 = initial_mean+ Uxtx[:,0]*up_width*0.5
eig2 = initial_mean+ Uxtx[:,1]*up_height*0.5
# ax1.plot([initial_mean[0],eig1[0]],[initial_mean[1],eig1[1]])
ax1.annotate('',xy=eig1,xytext=initial_mean,
             arrowprops=dict(arrowstyle="->"),
            horizontalalignment='center', verticalalignment='center')
ax1.annotate('',xy=eig2,xytext=initial_mean,
             arrowprops=dict(arrowstyle="->"),
            horizontalalignment='center', verticalalignment='center')
ax1.set_title('Rotate the Covariance')


eig_dir1 = lamx*Uxtx[1,0]/Uxtx[0,0]
eig_dir2 = lamx*Uxtx[1,1]/Uxtx[0,1]
ax2.plot(lamx,eig_dir1,color='silver')
ax2.plot(lamx,eig_dir2,color='silver')
ax2.set_ylim(lamy[0],lamy[-1])

ax2.set_title('Shift the Mean')
ax2.annotate('',xy=initial_mean,xytext=(0,0),
             arrowprops=dict(arrowstyle="->"),
            horizontalalignment='center', verticalalignment='center')
ax2.annotate('',xy=initial_mean-debias,xytext=initial_mean,
             arrowprops=dict(arrowstyle="->"),
            horizontalalignment='center', verticalalignment='center')
ax2.annotate('',xy=update_mean,xytext=initial_mean-debias,
             arrowprops=dict(arrowstyle="->"),
            horizontalalignment='center', verticalalignment='center')

# set labels
ax1.set_xlabel('$\lambda_1$')
ax1.set_ylabel('$\lambda_2$')
ax2.set_xlabel('$\lambda_1$')

# figure name will be 
this_fig_name = 'fig_wobbly_dci_decomp.png'
fig_all_master[this_fig_name] = fig_dci_decomp

# # save just this figure
# fig_dci_decomp.savefig('../'+this_fig_name)


## Theoretical Figures

These figures are used to illustrate some of the theoretical concepts in Chapter 2 regarding the contour structure of $Q$ etc..

In [ ]:
# gets the lam2 values from lam1 values along q
def Qinv_contour(lam1,QmapA,q=0,b=y0):
    x1,x2 = QmapA[0],QmapA[1]
    lam2 = -(x1/x2)*lam1+(q-b)/x2
    return lam1, lam2

In [ ]:
# q-values to compute the contours over
q_contour_vals = np.linspace(-10,10,21)

# get relative likelihood values across contours
q_contour_obs_like = obs_dist_1D.pdf(q_contour_vals)
q_contour_pred_like = predict_dist_1D.pdf(q_contour_vals)

# define the contours
contours = []
for q_val in q_contour_vals:
    lam1,lam2 = Qinv_contour(lamx,locations[pointA],q=q_val)
    contours.append([lam1,lam2])

In [ ]:
# get normalized cmap for colors
min_prob = np.min([q_contour_obs_like,q_contour_pred_like])
max_prob = 0.5*np.max([q_contour_obs_like,q_contour_pred_like])
this_color_norm = plt.matplotlib.colors.Normalize(vmin=min_prob,vmax=max_prob)

# define cmap
this_cmap = plt.cm.get_cmap('Blues')
colors_pred = this_cmap(this_color_norm(q_contour_pred_like))
colors_obs = this_cmap(this_color_norm(q_contour_obs_like))


### Shows the Contours of $Q$

In [ ]:
fig_dci_contours, (ax1,ax2) = plt.subplots(1,2,subplot_kw={'aspect':'equal'})
fig_dci_contours.tight_layout()
for i,cont in enumerate(contours):
    ax1.plot(cont[0],cont[1],color=colors_pred[i])
    ax1.set_ylim(lamy[0],lamy[-1])
    ax1.set_title('Contours of $\pi_{predict}$')
    ax1.set_xlabel('$\lambda_1$')
    ax1.set_ylabel('$\lambda_2$')
    
    ax2.plot(cont[0],cont[1],color=colors_obs[i])
    ax2.set_ylim(lamy[0],lamy[-1])
    ax2.set_title('Contours of $\pi_{obs}$')
    ax2.set_xlabel('$\lambda_1$')
    #ax2.set_ylabel('$\lambda_2$')

# # save fig
fig_name = 'dci_contour_q.png'
fig_all_master[fig_name] = fig_dci_contours

# # save this figure only
# fig_dci_contours.savefig('../'+fig_name)

### Examples Comparing Initials and Predicted

We compare data consistent updates with

\begin{align}
\pi_{init}^A &\sim N(0,\sigma_0^2I)\\
\pi_{init}^B &\sim U^2\left[-\sigma_0^2,\sigma_0^2\right]\\
\pi_{init}^C &\sim U^2\left[-\frac{1}{c}\sigma_0^2,\frac{1}{c}\sigma_0^2\right]
\end{align}

Given that $X$ is the row vector representing point A, we compute the predicted distributions as:

\begin{align}
\pi_{predict}^A &\sim N(0,\sigma_0^2XX^T) \\
\pi_{predict}^B &\sim \text{trapezoid}(b_1+y_0,b_2+y_0,b_3+y_0,b_4+y_0) \\
\pi_{predict}^C &\sim \text{trapezoid}(c_1+y_0,c_2+y_0,c_3+y_0,c_4+y_0) \\
\end{align}

where the constants of the trapezoid distributions are:

\begin{align}
b_1 &= -(|x_1|+|x_2|)\sigma_0^2 \\
b_2 &= \min(|x_1|-|x_2|,|x_2|-|x_1|)\cdot\sigma_0^2 \\
b_3 &= \max(|x_1|-|x_2|,|x_2|-|x_1|)\cdot\sigma_0^2 \\
b_4 &= (|x_1|+|x_2|)\sigma_0^2 
\end{align}

and $c_i=(1/c)b_i$.

For our comparison of $B$ and $C$ we also use sampling and GKDE to estimate the predicted distributions.

In [ ]:
# class defined for 2d uniform random variables
class uniform2D:
    def __init__(self,pdf1,pdf2):
        self.pdf1 = pdf1
        self.pdf2 = pdf2
        
    def rvs(self,N):
        sample1 = self.pdf1.rvs(N)
        sample2 = self.pdf2.rvs(N)
        return np.stack([sample1,sample2],axis=1)
    
    def pdf(self,X):
        out = self.pdf1.pdf(X[:,0])*self.pdf2.pdf(X[:,1])
        return out

In [ ]:
# define the initial distributions
sig0 = np.sqrt(initial_cov[0,0])
csig0 = (1/2)*sig0
init_distA = sps.multivariate_normal(np.zeros(2),initial_cov)

init_distB_marg = sps.uniform(-sig0,2*sig0)
init_distB = uniform2D(init_distB_marg,init_distB_marg)

init_distC_marg = sps.uniform(-csig0,2*csig0)
init_distC = uniform2D(init_distC_marg,init_distC_marg)

# define the exact predicted distributions
pred_distA = sps.norm(0+y0,np.sqrt(predict_cov_1D))

# obtain the constants for trapezoidal distribution
trapzB_const = sig0*np.dot(np.array([[-1,-1],[1,-1],[-1,1],[1,1]]),
                             np.abs(locations[pointA]))
trapzB_const = np.sort(trapzB_const) # ensure the points are in ascending order

# shift constants and transform to scipy-stats parameter specification
tpzB = {'loc': trapzB_const[0]+y0,
       'scale': trapzB_const[-1]-trapzB_const[0]}
tpzB['c'] = (trapzB_const[1]+y0-tpzB['loc'])/tpzB['scale']
tpzB['d'] = (trapzB_const[2]+y0-tpzB['loc'])/tpzB['scale']

# final predicted distribution
pred_distB = sps.trapz(loc=tpzB['loc'],scale=tpzB['scale'],
                       c=tpzB['c'],d=tpzB['d'])

# get the constants for C
tpzC = {'loc': (csig0/sig0)*trapzB_const[0]+y0,
       'scale': (csig0/sig0)*(trapzB_const[-1]-trapzB_const[0])}
tpzC['c'] = ((csig0/sig0)*trapzB_const[1]+y0-tpzC['loc'])/tpzC['scale']
tpzC['d'] = ((csig0/sig0)*trapzB_const[2]+y0-tpzC['loc'])/tpzC['scale']

pred_distC = sps.trapz(loc=tpzC['loc'],scale=tpzC['scale'],
                       c=tpzC['c'],d=tpzC['d'])


In [ ]:
# create the sample distributions with M points
M_init = 50
B_init_sample = init_distB.rvs(M_init)
C_init_sample = init_distC.rvs(M_init)

# compute predicted sample
B_pred_sample = Q2to1(B_init_sample)
C_pred_sample = Q2to1(C_init_sample)

# compute kdes
approx_pred_kdeB = sps.gaussian_kde(B_pred_sample)
approx_pred_kdeC = sps.gaussian_kde(C_pred_sample)

In [ ]:
# qualitative check that the predicted distributions are satisfactory
thisq = np.linspace(-1,9,100)

# observed
plt.plot(thisq,obs_dist_1D.pdf(thisq),label='Obs',color='k',ls='--')

# predicted
plt.plot(thisq,pred_distA.pdf(thisq),label='Pred A')
plt.plot(thisq,pred_distB.pdf(thisq),label='Pred B')
plt.plot(thisq,pred_distC.pdf(thisq),label='Pred C')

# kdes
plt.plot(thisq,approx_pred_kdeB(thisq),label='~Pred B',color='xkcd:orange',ls='dotted')
plt.plot(thisq,approx_pred_kdeC(thisq),label='~Pred C',color='xkcd:green',ls='dotted')
plt.title('Predicted vs. Observed')
plt.legend()

In [ ]:
# define update A and update B
upA_dist = dci_update(init_distA,pred_distA,obs_dist_1D,Q2to1)
upB_dist = dci_update(init_distB,pred_distB,obs_dist_1D,Q2to1)

# attempt approximate dist
approx_upB_dist = dci_update(init_distB,approx_pred_kdeB,obs_dist_1D,Q2to1)
approx_upC_dist = dci_update(init_distC,approx_pred_kdeC,obs_dist_1D,Q2to1)

In [ ]:
# evaluate points for 3D plot
Z_upA = upA_dist.pdf(eval_lamXY.T).reshape(lamX.shape)
Z_upB = upB_dist.pdf(eval_lamXY.T).reshape(lamX.shape)

# points for contour line
q_val = 3.5
contX,contY = Qinv_contour(lamx,locations[pointA],q=q_val)
Z_contA = upA_dist.pdf(np.array([contX,contY]).T)#/obs_dist.pdf(q_val)
Z_contB = upB_dist.pdf(np.array([contX,contY]).T)#/obs_dist.pdf(q_val)

# points for approx distribution plot
approx_Z_upB = approx_upB_dist.pdf(eval_lamXY.T).reshape(lamX.shape)
approx_Z_upC = approx_upC_dist.pdf(eval_lamXY.T).reshape(lamX.shape)

In [ ]:
# make plot
fig_dci_init_contour = plt.figure()
fig_dci_init_contour.set_figwidth(14)
fig_dci_init_contour.set_figheight(12)

spec_test = fig_dci_init_contour.add_gridspec(2,2,width_ratios=[1,3])
axes3D = [fig_dci_init_contour.add_subplot(2,2,j,projection='3d') for j in [1,3]]
axes = [fig_dci_init_contour.add_subplot(2,2,j) for j in [2,4]]

theseZs = [Z_upA,Z_upB]
thisq = np.linspace(-3,9,200)
thispdf = [pred_distA,pred_distB]
thiscontZ = [Z_contA,Z_contB]
titles = ['Dist A','Dist B']

for i in np.arange(2):
    axes3D[i].plot_surface(lamX,lamY,theseZs[i],alpha=0.68,cmap='GnBu',
                          vmin=-0.025)
    axes3D[i].set_title('Update {}'.format(titles[i]))
    axes3D[i].plot(contX,contY,thiscontZ[i],linewidth=5,zorder=0)
    
    axes[i].plot(thisq,obs_dist_1D.pdf(thisq),label='Obs',
                 color='xkcd:maroon',alpha=0.5,ls='--')
    axes[i].plot(thisq,thispdf[i].pdf(thisq),label='Pred')
    axes[i].plot(q_val,obs_dist_1D.pdf(q_val),'o',color='C1',
                    label='$q={}$'.format(q_val))
    if i==1:
        axes[i].set_xlabel('$q$')
    axes[i].legend()
    axes[i].set_title('Predict {}'.format(titles[i]))
    
    # 3D plotting labels
    axes3D[i].view_init(30,-65)
    axes3D[i].set_xlabel('$\lambda_1$')
    axes3D[i].set_ylabel('$\lambda_2$')
    
#     for j,ax in enumerate(ax_row):
#         if j==0:
#             ax.remove()
#             ax.plot_surface(lamX,lamY,Z_upA)
#         ax.set_title('{},{}'.format(i,j))

# # save fig
fig_name = 'dci_3D_init.png'
fig_all_master[fig_name] = fig_dci_init_contour

# # save this figure only
# fig_dci_init_contour.savefig('../'+fig_name)

In [ ]:
# make plot
fig_dci_pred_contour = plt.figure()
fig_dci_pred_contour.set_figwidth(14)
fig_dci_pred_contour.set_figheight(12)
spec_test = fig_dci_pred_contour.add_gridspec(2,2,width_ratios=[1,3])
axes3D = [fig_dci_pred_contour.add_subplot(2,2,j,projection='3d') for j in [1,3]]
axes = [fig_dci_pred_contour.add_subplot(2,2,j) for j in [2,4]]

theseZs = [approx_Z_upB,approx_Z_upC]
this_vmax = np.max(approx_Z_upB)
thisq = np.linspace(-3,9,200)
thispdf = [pred_distB,pred_distC]
thisAPPROXpdf = [approx_pred_kdeB,approx_pred_kdeC]
thiscontZ = [Z_contA,Z_contB]
titles = ['Dist B','Dist C']

for i in np.arange(2):
    axes3D[i].plot_surface(lamX,lamY,theseZs[i],alpha=0.8,cmap='GnBu',
                           vmin=-0.025,vmax=this_vmax)
    axes3D[i].set_title('Update {}'.format(titles[i]))
#     axes3D[i].plot(contX,contY,thiscontZ[i],linewidth=5)
    
    axes3D[i].view_init(30,-65)
    axes3D[i].set_xlabel('$\lambda_1$')
    axes3D[i].set_ylabel('$\lambda_2$')

    
    axes[i].plot(thisq,obs_dist_1D.pdf(thisq),label='Obs',
                 color='xkcd:maroon',alpha=0.5,ls='--')
    axes[i].plot(thisq,thispdf[i].pdf(thisq),label='Pred',alpha=0.7)
    axes[i].plot(thisq,thisAPPROXpdf[i].pdf(thisq),label='Approx Pred')
    axes[i].plot(q_val,obs_dist_1D.pdf(q_val),'o',color='C1',
                    label='$q={}$'.format(q_val))
    if i==1:
        axes[i].set_xlabel('$q$')

    axes[i].legend()
    axes[i].set_title('Predict {}'.format(titles[i]))
    
# # save fig
fig_name = 'dci_3D_pred_violate.png'
fig_all_master[fig_name] = fig_dci_pred_contour

# # save this figure only
# fig_dci_pred_contour.savefig('../'+fig_name)

#### Accept-Reject Algorithm

Here we do this to approximately compute $r$.

In [ ]:
# Approximate r-value
np.mean(obs_dist_1D.pdf(B_pred_sample)/approx_pred_kdeB(B_pred_sample))

In [ ]:
# Approximate r-value
np.mean(obs_dist_1D.pdf(C_pred_sample)/approx_pred_kdeC(C_pred_sample))

# Parameter Estimation Figures

In this section we concern ourselves with the "fixed plate" scenario. In other words, this is the case where we want to estimate the slope (and uncertainty) of $\lambda$, but $\lambda$ is not subject to aleotoric uncertainty.

In [ ]:
# used to compute orthogonal data-informed directions
def Qinv_ortho_contour(lam1,QmapA,point=np.zeros(2)):
    x1,x2 = QmapA[0],QmapA[1]
    ortho_slope = x2/x1
    lam2 = ortho_slope*(lam1-point[0])+point[1]
    return lam1, lam2
    

### Theoretical Figures

Note that here we don't actually take random samples but fix some example samples for the purposes of plotting.

We just want to illustrate the idea with certain samples converging.

In [ ]:
# true slope
print(lam_mean)
ex_obs_dist = sps.norm()

N_list = [10,20,30,60,np.inf]

this_mean = obs_mean[pointA]
print(obs_mean[pointA])

sample_dict = {}
for N in N_list:
    sample_dict[N] = {}
    if N == np.inf:
        sample_dict[N]['mean'] = obs_mean[pointA]
        sample_dict[N]['cov'] = obs_cov_1D
    else:
        if N==10:
            this_sample = obs_mean-1/np.sqrt(N)*3
        elif N==20:
            this_sample = obs_mean-1/np.sqrt(N)*3*np.random.uniform(0.7,1)
        elif N==30:
            this_sample = obs_mean+1/np.sqrt(N)*3*np.random.uniform(0.5,1.1)
        else:
            this_sample = obs_mean+1/np.sqrt(N)*3*0.5
        sample_dict[N]['mean'] = np.mean(this_sample)
        
    # define the linear function using polynomial
    lam1,lam2 = Qinv_contour(lamx,locations[pointA],q=sample_dict[N]['mean'])
    this_contour_line = np.polynomial.polynomial.Polynomial.fit(lam1,lam2,1)
    sample_dict[N]['line_func'] = this_contour_line

In [ ]:
fig_dci_param_schema,ax = plt.subplots(1)
fig_dci_param_schema.set_figwidth(5.5)
fig_dci_param_schema.tight_layout()

midline = sample_dict[np.inf]['line_func'](lamx)

# get the color cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for i,N in enumerate(N_list):
    this_qbar = sample_dict[N]['mean']
    lam1,lam2 = Qinv_contour(lamx,locations[pointA],q=this_qbar)
    plt.plot(lam1,lam2,color=colors[i])
    contour_label = '$\\bar{{ q }}_{{ {} }}$'.format(N)
    if N==np.inf:
        contour_label = '$\mu_q$'
    ybound = np.argwhere(np.diff(np.sign(lam2)))
    xbound = np.argwhere(np.diff(np.sign(lam1-2)))
    this_ind = np.min([xbound,ybound])
    
    if lam2[-1]>midline[-1]:
        print('This is above mean')
        ups = 0.3
        sides = 0.05
        valign = 'bottom'
    else:
        ups = 0.05
        sides = 0.1
        valign = 'center'
    
    locs = [lam1[this_ind]-sides,lam2[this_ind]+ups]
    
    plt.annotate(contour_label,xy=locs,ha='right',va=valign,color=colors[i])
    
# the mean spot
plt.plot(lam_mean[0],lam_mean[1],marker='^',color='k',ms=8)
plt.annotate('$\lambda^{\dagger}$',lam_mean+np.array([-0.07,0]),va='top',ha='right')

# the least squares point
# lam1_ortho,lam2_ortho = Qinv_ortho_contour(lamx,locations[pointA])
# plt.plot(lam1_ortho,lam2_ortho,color='k',ls='--')

# the dci point
this_init_mean = [-0.75,1]
lam1_DCI,lam2_DCI = Qinv_ortho_contour(lamx,locations[pointA],
                                           point=this_init_mean)
this_DCI_line = np.polynomial.polynomial.Polynomial.fit(lam1_DCI,lam2_DCI,1)

plt.plot(lam1_DCI,lam2_DCI,color='k',ls='--')
plt.plot(*this_init_mean,color='k',marker='o',ms=7.5)
plt.annotate('$\mu_0$',this_init_mean+np.array([0,-0.05]),va='top')

for i,N in enumerate(N_list):
    this_lamx = np.linspace(-1,2,200)
    contour_ys = sample_dict[N]['line_func'](this_lamx)
    DCI_ys = this_DCI_line(this_lamx)
    this_ind = np.argwhere(np.diff(np.sign(contour_ys-DCI_ys)))
    this_label = '$\\lambda^{MUD}$' if N==np.inf else None
    plt.plot(this_lamx[this_ind],DCI_ys[this_ind],
             color=colors[i],marker='D',ms=7.5,label=this_label,
             linestyle='None')

plt.legend(loc='upper right')
plt.xlim(-1,2)
plt.ylim(np.array(plt.xlim())+1)
ax.set_aspect('equal')
ax.set_xlabel('$\lambda_1$')
ax.set_ylabel('$\lambda_2$')
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('DCI Parameter Estimation')

# # save fig
fig_name = 'fig_dci_param_schema.png'
fig_all_master[fig_name] = fig_dci_param_schema

# # save just this fig
# fig_dci_param_schema.savefig('../'+fig_name)

### 2-Figure: Compare Covariance Structures DCI

Here we compare the covariance structure using regular DCI versus changing the quantity of interest to the distribution of sample means (or the $Q_{WME}$ as defined in the paper).

In [ ]:
# for getting multiple update covariances quickly
def get_dci_cov(init_cov, obs_cov, QmapA):
    # general formulas for 2-to-1
    XTX = np.outer(QmapA.T,QmapA)
    predict_cov = np.linalg.multi_dot([QmapA.T,init_cov,QmapA])
    q_cov_diff = 1/predict_cov*(1-obs_cov/predict_cov)
    update_cov = init_cov-q_cov_diff*np.linalg.multi_dot([init_cov,XTX,init_cov])
    return update_cov

In [ ]:
sps.chi2.ppf(1-0.15,df=2)

In [ ]:
# for making the ellipses easier
def make_cov_ellipse(center,cov,area=0.9,ellipse_args={}):
    # sets an area for the circles that includes area% of data
    this_area = sps.chi2.ppf(area,df=2) 
    this_center = center
    
    # eigen decomp covariance for ellipse parameters
    this_eig_decomp = np.linalg.eig(cov)
    
    # eigen value widths and heights
    this_eig0, this_eig1 = this_eig_decomp[0]
    this_width = 2*np.sqrt(this_area/2*this_eig0)
    this_height = 2*np.sqrt(this_area/2*this_eig1)
    
    # rotation
    this_eig_vec = this_eig_decomp[1][:,0]
    rot_angle = np.arctan(this_eig_vec[1]/this_eig_vec[0])*180/np.pi

    this_ellipse = patches.Ellipse(xy=this_center,angle=rot_angle,
                               width=this_width,height=this_height,
                               **ellipse_args)
    return this_ellipse

In [ ]:
fig_dci_compare_cov,axes = plt.subplots(1,2)
fig_dci_compare_cov.set_figwidth(8)

midline = sample_dict[np.inf]['line_func'](lamx)

these_titles = ['QoI Map $Q(\lambda)$','QoI Map $\\widebar{Q}(\lambda)$']
# get the color cycle
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for j,ax in enumerate(axes):
    for i,N in enumerate(N_list):
        if N==30:
            continue
        this_qbar = sample_dict[N]['mean']
        lam1,lam2 = Qinv_contour(lamx,locations[pointA],q=this_qbar)
        ax.plot(lam1,lam2,color=colors[i])
        contour_label = '$\\bar{{ q }}_{{ {} }}$'.format(N)
        if N==np.inf:
            contour_label = '$\mu_q$'
        ybound = np.argwhere(np.diff(np.sign(lam2)))
        xbound = np.argwhere(np.diff(np.sign(lam1-2)))
        this_ind = np.min([xbound,ybound])

        if lam2[-1]>midline[-1]:
            print('This is above mean')
            ups = 0.3
            sides = 0.05
            valign = 'bottom'
        else:
            ups = 0.05
            sides = 0.1
            valign = 'center'

        locs = [lam1[this_ind]-sides,lam2[this_ind]+ups]

        ax.annotate(contour_label,xy=locs,ha='right',va=valign,color=colors[i])

#     # the mean spot
#     ax.plot(lam_mean[0],lam_mean[1],marker='^',color='k',ms=8)
#     ax.annotate('$\lambda^{\dagger}$',lam_mean+np.array([-0.07,0]),va='top',ha='right')

    # the least squares point
    # lam1_ortho,lam2_ortho = Qinv_ortho_contour(lamx,locations[pointA])
    # plt.plot(lam1_ortho,lam2_ortho,color='k',ls='--')

    # the dci point
    this_init_mean = [-0.75,1]
    lam1_DCI,lam2_DCI = Qinv_ortho_contour(lamx,locations[pointA],
                                               point=this_init_mean)
    this_DCI_line = np.polynomial.polynomial.Polynomial.fit(lam1_DCI,lam2_DCI,1)

    ax.plot(lam1_DCI,lam2_DCI,color='k',ls='--')
    ax.plot(*this_init_mean,color='k',marker='o',ms=7.5)
    ax.annotate('$\mu_0$',this_init_mean+np.array([0,-0.05]),va='top')

    for i,N in enumerate(N_list):
        if N==30:
            continue
        this_lamx = np.linspace(-1,2,200)
        contour_ys = sample_dict[N]['line_func'](this_lamx)
        DCI_ys = this_DCI_line(this_lamx)
        this_ind = np.argwhere(np.diff(np.sign(contour_ys-DCI_ys)))
        this_label = 'Update Cov' if N==10 else None
        ax.plot(this_lamx[this_ind],DCI_ys[this_ind],
                 color=colors[i],marker='D',ms=7.5,label=this_label,
                 linestyle=':')

        if N==np.inf:
            continue
        # make ellipse
        this_obs_cov = obs_cov_1D/3
        this_obs_cov /= 1 if j==0 else np.sqrt(N)
        this_center = np.array([this_lamx[this_ind],DCI_ys[this_ind]])
        this_cov = get_dci_cov(np.eye(2),this_obs_cov,locations[pointA])
        this_cov_ellipse = make_cov_ellipse(this_center,this_cov,
                                            area=0.8,ellipse_args=
                                           {'edgecolor': colors[i],'fc': "None",
                                            'alpha': 1,'ls':':','linewidth':2})
        ax.add_artist(this_cov_ellipse)

    ax.legend(loc='upper right')
    ax.set_xlim(-1,2)
    ax.set_ylim(np.array(ax.get_xlim())+1)
    ax.set_aspect('equal')
    ax.set_xlabel('$\lambda_1$')
    ax.set_ylabel('$\lambda_2$')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(these_titles[j])

# # save fig
fig_name = 'fig_dci_param_QvsQbar.png'
fig_all_master[fig_name] = fig_dci_compare_cov

# # save just this fig
# fig_dci_compare_cov.savefig('../'+fig_name)

### 4-Figure: Compare Parameter Estimations

Here we create a figure to compare the Bayesian MAP estimates versus the DCI MUD estimates.

In [ ]:
# define a function to get the posterior mu
def get_posterior_mu(init_params,QmapA,qbar,obs_cov,y0=3):
    mu0, cov0 = init_params[0],init_params[1]
    
    # covariance
    pred_cov = np.linalg.multi_dot([QmapA.T,cov0,QmapA])
    total_cov = pred_cov + obs_cov
    
    # data diff
    data_diff = qbar-(np.dot(QmapA,mu0) + y0)
    
    # posterior
    post_mu = mu0+(1/total_cov)*np.dot(np.dot(cov0,QmapA.T),data_diff)
    return post_mu

In [ ]:
# definte this example domain
lam1_pe = np.linspace(0,5,300)
lam2_pe = np.linspace(0,5,300)

# sensor location
pe_sens_loc = np.array([0.5,1.1])

# initial params
pe_mu0 = np.array([2.25,2])
pe_cov0 = np.eye(2)

In [ ]:
fig_param_est_compares , axes = plt.subplots(2,2,subplot_kw={'aspect':'equal'})
fig_param_est_compares.set_figheight(8)
plt.suptitle('Comparison of Parameter Estimates')
# fig_param_est_compares.tight_layout(pad=2)
these_titles = [[None,None],[None,None]]
for i,ax_row in enumerate(axes):
    for j,ax in enumerate(ax_row):
        ax.autoscale(False)
#         ax.plot(lam1_pe,lam2_pe,visible=True)
        
        # the solution contour
        out = np.array(Qinv_contour(lam1_pe,pe_sens_loc,q=8))
        ax.plot(*out,color='xkcd:maroon',alpha=0.4,ls='--')
        ax.plot(out[0][225], out[1][225],marker='^',ms=8,ls='--',
                color='xkcd:maroon')
        ax.annotate('$\lambda^{\dagger}$',out[:,225]+np.array([0,0.05]),
                    color='xkcd:maroon',va='bottom')
        
        # the least squares solution set
        ls_line = np.array(Qinv_ortho_contour(lam1_pe,pe_sens_loc))
        ax.plot(*ls_line,color='C0',ls='--',alpha=0.6)
        
        
        # the dci and Bayes solution set
        dci_line = np.array(Qinv_ortho_contour(lam1_pe,pe_sens_loc,pe_mu0))
        ax.plot(*dci_line,color='C1',ls='--',alpha=0.6)
        ax.plot(*pe_mu0,marker='o',ms=6,ls='--',
                color='k')
        ax.annotate('$\mu_0$',pe_mu0+np.array([-0.2,0]),
                    color='k',va='center',ha='right')
        
        
        # columns vary by sample size
        Nsize = 2 if j==0 else 30
        q_var = 3/np.sqrt(Nsize)
        
        # top row
        if i==0:
            # plot the sample means contour
            q_mean = 5.25
            q_bar_out = np.array(Qinv_contour(lam1_pe,pe_sens_loc,q=q_mean))
            ax.plot(*q_bar_out,color='xkcd:red',alpha=1,ls='-')
            
        else:
            q_mean = 7.5
            # plot the sample means contour
            q_bar_out = np.array(Qinv_contour(lam1_pe,pe_sens_loc,q=q_mean))
            ax.plot(*q_bar_out,color='xkcd:red',alpha=1,ls='-')
        
        these_titles[i][j] = '$N={}$, $\\bar{{ q }}={}$'.format(Nsize,q_mean)
        
        # label the ls solution
        ls_ind = np.argmin(np.abs(ls_line[1]-q_bar_out[1]))
        ax.plot(*ls_line[:,ls_ind], marker='D',ms=6, color='k')
        ax.annotate('$\lambda^{LS}$',ls_line[:,ls_ind]+np.array([-0.2,-0.2]),
                color='k',va='top',ha='center')
        
        # plot posterior mean
        # get posterior mean
        lam_post = get_posterior_mu([pe_mu0,pe_cov0],pe_sens_loc,q_mean,q_var)
        ax.plot(*lam_post, marker='D',ms=6, color='k')
        ups_adj = -0.3 if j==1 and i==1 else -0.1
        ax.annotate('$\lambda^{MAP}$',lam_post+np.array([0.2,ups_adj]),
                color='k',va='center',ha='left')
        
        # plot the dci solution
        # get posterior mean
        dci_ind = np.argmin(np.abs(dci_line[1]-q_bar_out[1]))
        ax.plot(*dci_line[:,dci_ind], marker='D',ms=6, color='k')
        ups_adj = 0.1 if i==1 else -0.2
        valign = 'bottom' if i==1 else 'top'
        ax.annotate('$\lambda^{MUD}$',dci_line[:,dci_ind]+np.array([0.2,ups_adj]),
                color='k',va=valign,ha='center')
        
        # labels and stuff
        ax.set_xlim([lam1_pe[0],lam1_pe[-1]])
        ax.set_ylim([lam2_pe[0],lam2_pe[-1]])
#         ax.legend()
        ax.set_title(these_titles[i][j])
fig_param_est_compares.tight_layout()     

# # save fig
fig_name = 'fig_dci_param_compare.png'
fig_all_master[fig_name] = fig_param_est_compares

# # save just this fig
# fig_param_est_compares.savefig('../'+fig_name)

### 2-Figure: Compare Covariances

Here we show that if the initial / prior covariance is biased, the Bayesian MAP estimate is more influenced by the bias of the covariance than the DCI MUD point.

In [ ]:
# biased covariances
# define the observed covariance
pe_cov_rot = -np.pi/4 # rotation in radians
pe_cov_dir = np.array([[np.cos(pe_cov_rot),-np.sin(pe_cov_rot)],
                    [np.sin(pe_cov_rot),np.cos(pe_cov_rot)]])
pe_cov_eig = 0.25*np.array([5,1])*np.eye(2)
pe_init_cov = np.linalg.multi_dot([pe_cov_dir,pe_cov_eig,pe_cov_dir.T])

# get the eigen decomposition
pe_eig_decomp = np.linalg.eig(pe_init_cov)
print(pe_eig_decomp)

In [ ]:
# figure of covariance influencing MAP point
fig_param_cov_influences , ax_row = plt.subplots(1,2,subplot_kw={'aspect':'equal'})
fig_param_cov_influences.set_figwidth(6)

these_titles = ['Spherical Covariance','Biased Covariance']
for j,ax in enumerate(ax_row):
    ax.autoscale(False)
#         ax.plot(lam1_pe,lam2_pe,visible=True)
    
    if j==0:
        # initial ellipse
        area90 = 4.605 # sets an area for the circles that includes 90% of data
        init_center = pe_mu0
        init_width = 2*np.sqrt(area90/2*pe_init_cov[0,0])
        init_ellipse = patches.Ellipse(xy=init_center,
                                       width=init_width,height=init_width,
                                       edgecolor='b',fc="None",alpha=0.6)
        ax.add_artist(init_ellipse)
        
        # find the direction
        dci_line = np.array(Qinv_ortho_contour(lam1_pe,pe_sens_loc,pe_mu0))
        dci_ind = np.argmin(np.abs(dci_line[1]-q_bar_out[1]))
        
        this_direc = dci_line[:,dci_ind]-init_center
        this_vec = np.array([this_direc[0],this_direc[1]])
        this_vec_normed = this_vec/np.linalg.norm(this_vec)
        this_vec_ortho = np.array([this_vec_normed[1],-this_vec_normed[0]])
        
        eig1 = init_center+ this_vec_normed*init_width*0.5
        eig2 = init_center- this_vec_ortho*init_width*0.5
        
        ax.annotate('',xy=eig1,xytext=init_center,
                 arrowprops=dict(arrowstyle="->",color='b',alpha=0.4),
                horizontalalignment='center', verticalalignment='center')
        ax.annotate('',xy=eig2,xytext=init_center,
                     arrowprops=dict(arrowstyle="->",color='b',alpha=0.4),
                    horizontalalignment='center', verticalalignment='center')
        
        
    else:
        this_eig0, this_eig1 = pe_eig_decomp[0]
        new_width = 2*np.sqrt(area90/2*this_eig0)
        new_height = 2*np.sqrt(area90/2*this_eig1)
        this_eig_vec = pe_eig_decomp[1][:,0]
        rot_angle = np.arctan(this_eig_vec[1]/this_eig_vec[0])*180/np.pi

        init_ellipse_ROT = patches.Ellipse(xy=init_center,angle=rot_angle,
                                       width=new_width,height=new_height,
                                       edgecolor='b',fc="None",alpha=0.6)
        ax.add_artist(init_ellipse_ROT)
        
        # draw eigen arrows
        eig1 = init_center- pe_eig_decomp[1][:,0]*new_width*0.5
        eig2 = init_center+ pe_eig_decomp[1][:,1]*new_height*0.5

        ax.annotate('',xy=eig1,xytext=init_center,
                 arrowprops=dict(arrowstyle="->",color='b',alpha=0.4),
                horizontalalignment='center', verticalalignment='center')
        ax.annotate('',xy=eig2,xytext=init_center,
                     arrowprops=dict(arrowstyle="->",color='b',alpha=0.4),
                    horizontalalignment='center', verticalalignment='center')
    
    # the solution contour
    out = np.array(Qinv_contour(lam1_pe,pe_sens_loc,q=8))
    ax.plot(*out,color='xkcd:maroon',alpha=0.4,ls='--')
    ax.plot(out[0][225], out[1][225],marker='^',ms=8,ls='--',
            color='xkcd:maroon')
    ax.annotate('$\mu^{\dagger}$',out[:,225]+np.array([0,0.05]),
                color='xkcd:maroon',va='bottom')

    # the dci and Bayes solution set
    dci_line = np.array(Qinv_ortho_contour(lam1_pe,pe_sens_loc,pe_mu0))
    ax.plot(*dci_line,color='C1',ls='--',alpha=0.6)
    ax.plot(*pe_mu0,marker='o',ms=6,ls='--',
            color='k')
    ax.annotate('$\mu_0$',pe_mu0+np.array([-0.2,0]),
                color='k',va='center',ha='right')


    # columns vary by sample size
    Nsize = 30
    q_var = 3/np.sqrt(Nsize)


    q_mean = 7.5
    # plot the sample means contour
    q_bar_out = np.array(Qinv_contour(lam1_pe,pe_sens_loc,q=q_mean))
    ax.plot(*q_bar_out,color='xkcd:red',alpha=1,ls='-')

    

    # plot posterior mean
    # get posterior mean
    this_cov = pe_cov0 if j==0 else pe_init_cov
    lam_post = get_posterior_mu([pe_mu0,this_cov],pe_sens_loc,q_mean,q_var)
    ax.plot(*lam_post, marker='D',ms=6, color='k')
    ups_adj = -0. if j==1 else 0.2
    ax.annotate('$\lambda^{MAP}$',lam_post+np.array([-0.2,ups_adj]),
            color='k',va='center',ha='right')

    # plot the dci solution
    # get posterior mean
    dci_ind = np.argmin(np.abs(dci_line[1]-q_bar_out[1]))
    ax.plot(*dci_line[:,dci_ind], marker='D',ms=6, color='k')
    ups_adj = 0.1 if i==1 else -0.2
    valign = 'bottom' if i==1 else 'top'
    ax.annotate('$\lambda^{MUD}$',dci_line[:,dci_ind]+np.array([0.2,ups_adj]),
            color='k',va=valign,ha='center')

    # labels and stuff
    ax.set_xlim([lam1_pe[0],lam1_pe[-1]])
    ax.set_ylim([lam2_pe[0],lam2_pe[-1]])
#         ax.legend()
    ax.set_title(these_titles[j])
fig_param_cov_influences.tight_layout()     

# # save fig
fig_name = 'fig_dci_param_bias_cov.png'
fig_all_master[fig_name] = fig_param_cov_influences

# fig_param_cov_influences.savefig('../'+this_fig_name)

# Save all Figs

In [ ]:
# make sure all the figures are there
for nf,key in enumerate(fig_all_master.keys()):
    print(nf,' ',key)

In [ ]:
# check individual figures
check_name = None
if check_name == None:
    print()
else:
    display(fig_all_master[check_name])

In [ ]:
# # # save all figs
# for figfilename in fig_all_master:
#     fig_all_master[figfilename].savefig('../'+figfilename,
#                                         dpi=250,bbox_inches='tight')